# Pose Pipeline

This notebook walks through video loading, pose extraction, visualization and data export.

In [1]:
import sys
from pathlib import Path
from IPython.display import Video
import pandas as pd
import numpy as np
import cv2

#Import the pose_pipeline model
from models.pose_pipeline import PosePipeline

# Add project root to Python path so we can import ml module
project_root = Path(__file__).parent.parent if '__file__' in globals() else Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


In [2]:
#Define video path
video_path = "../data/raw/videos/sample_good_shot.mp4"

In [3]:
#Instantiate PosePipeline
pose_pipeline = PosePipeline()

Model loaded successfully.


In [ ]:
#Processes the video and returns a pandas dataframe containing keypoints (specific body parts  / joints), confidence level and x,y coordinates.
results = pose_pipeline.process_video(video_path)


**Data Export**

The following dataframe has the following columns:
- frame: Represents the frame 0 to n.
- keypoint: Integer representation of a body part, ex. 1 is Left eye. 
- x: x coordinate in the frame of the keypoint.
- y: y coordinate in the frame of the keypoint.
- confidence: confidence level.

In [5]:
print(results)

      frame  keypoint           x           y  confidence
0         0         0  932.666138  507.891052    0.944163
1         0         1  944.647217  500.020966    0.944025
2         0         2  926.966248  499.110352    0.698841
3         0         3  971.585083  510.051300    0.921639
4         0         4  928.234619  508.446075    0.301193
...     ...       ...         ...         ...         ...
1134     66        12  909.207886  676.360657    0.994243
1135     66        13  960.401855  809.508606    0.991680
1136     66        14  895.763428  811.693481    0.982389
1137     66        15  993.740784  926.484985    0.942214
1138     66        16  913.520874  937.948608    0.911847

[1139 rows x 5 columns]


**Visualization Using Data**

Use the exported data to create a pose overlay visualization of the shooter.

In [12]:

# COCO skeleton connections
SKELETON = [
    # arms
    (5, 7), (7, 9),      # left arm
    (6, 8), (8, 10),     # right arm

    # shoulders
    (5, 6),

    # torso (THIS IS WHAT YOU WERE MISSING)
    (5, 11),             # left shoulder → left hip
    (6, 12),             # right shoulder → right hip
    (11, 12),            # hips
    (5, 6),              # shoulders (top of torso)

    # legs
    (11, 13), (13, 15),  # left leg
    (12, 14), (14, 16),  # right leg
]


def render_shooter_video(video_path, pose_df, output_path):
    cap = cv2.VideoCapture(video_path)

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS) or 30

    out = cv2.VideoWriter(
        output_path,
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (width, height)
    )

    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_data = pose_df[pose_df["frame"] == frame_idx]

        # Draw joints
        for _, row in frame_data.iterrows():
            x, y = int(row.x), int(row.y)
            cv2.circle(frame, (x, y), 4, (0, 0, 255), -1)

        # Draw bones
        for a, b in SKELETON:
            pa = frame_data[frame_data["keypoint"] == a]
            pb = frame_data[frame_data["keypoint"] == b]
            if not pa.empty and not pb.empty:
                x1, y1 = int(pa.x.values[0]), int(pa.y.values[0])
                x2, y2 = int(pb.x.values[0]), int(pb.y.values[0])
                cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()


**Rendering**

The video does not display on the IDE, but works on an MP4 player.

In [15]:
#Render the video with pose overlay

df = pose_pipeline.process_video(video_path)

render_shooter_video(
    video_path,
    df,
    "../data/processed/shooter_only3.mp4"
)


video 1/1 (frame 1/67) /Users/terr/Desktop/dsci_basketball/basketball-optimizer/notebooks/../data/raw/videos/sample_good_shot.mp4: 416x640 2 persons, 47.3ms
video 1/1 (frame 2/67) /Users/terr/Desktop/dsci_basketball/basketball-optimizer/notebooks/../data/raw/videos/sample_good_shot.mp4: 416x640 3 persons, 45.1ms
video 1/1 (frame 3/67) /Users/terr/Desktop/dsci_basketball/basketball-optimizer/notebooks/../data/raw/videos/sample_good_shot.mp4: 416x640 4 persons, 42.6ms
video 1/1 (frame 4/67) /Users/terr/Desktop/dsci_basketball/basketball-optimizer/notebooks/../data/raw/videos/sample_good_shot.mp4: 416x640 3 persons, 50.0ms
video 1/1 (frame 5/67) /Users/terr/Desktop/dsci_basketball/basketball-optimizer/notebooks/../data/raw/videos/sample_good_shot.mp4: 416x640 2 persons, 40.1ms
video 1/1 (frame 6/67) /Users/terr/Desktop/dsci_basketball/basketball-optimizer/notebooks/../data/raw/videos/sample_good_shot.mp4: 416x640 3 persons, 35.4ms
video 1/1 (frame 7/67) /Users/terr/Desktop/dsci_basketbal

In [16]:

Video(
    "../data/processed/shooter_only2.mp4",
)


The processed video should be in the data/processed directory.